Save diff dataframe to table

In [1]:
import requests
import datetime
import time
import os
import csv
import pandas as pd
# from os import environ
from sqlalchemy import MetaData, create_engine, asc
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

import psycopg2
from psycopg2 import Error

from flask import Flask, jsonify, request
from flask_cors import cross_origin, CORS
from models.models import Base

from models.models import ExpensesRaw, Filenames
# from endpointClasses.resources import Resources
#
from sqlalchemy import Column, Integer, BigInteger, String, Text, DateTime, \
    Float, Boolean, func, ForeignKeyConstraint, Index, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method
from datetime import datetime, timedelta, timezone
from geoalchemy2 import Geography, Geometry


In [2]:
run_all_flag = True

In [3]:
### Setup the application
app = Flask(__name__)

# Wrap CORS around the app so that the server does not block machine to machine
# or browser based requests
CORS(app)

In [4]:
# Engine below for Google Cloud PostgreSQL access.
engine = create_engine('postgresql+psycopg2://postgres:5413CrossFit2018@34'
                       '.70.40.80/transgov')

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [5]:
col_names = ["Ministry", "Position", "Name", "Type", "Category", "Date", "Amount",
         "Description", "Receipt 1", "Receipt 2", "Receipt 3"]

In [6]:
# download  csv file
# Only do this once per day
print('Downloading .....')
df = pd.read_csv(r'https://expenses.alberta.ca/DownloadData.aspx?type=csv'
                   r'&d=IsVE/OcdpNZJ5rBbvji3qw', names=col_names,
                     low_memory=False, parse_dates=['Date'])

# Saving the dataframe
print('Saving to CSV ...')
filename = 'expenses/' + str( time.strftime('%Y%m%d') ) + ".csv"
df.to_csv(filename)
print('done...')

Saving to CSV ...
done...


In [7]:
print('Reading from file...')
# filename = "/" + filename
df=pd.read_csv(filename, names=col_names, low_memory=False, skiprows=2)
df_size = len(df.index)
print('Done...', df_size)

Reading from file...
Done... 539390


In [8]:
# compare the current download to the most previous download so that we can identify
# the changes and only process those.
# 1. Check file sizes
# 2. iterate through DF1 and lookup record in DF2
# 3.   if record exists then do nothing
# 4.   is this new record or a changed one? (hard to determine....we may have to manually see if dups become a problem)

# 4.   otherwise add record to database

print('Reading from previous file...')
# get the most recent entry in the table FileNames
prevFile = pd.read_sql(
        session.query(Filenames).statement, session.bind)

previousFile = prevFile._get_value(0,'name')

# open the file as a dataframe
prev_df = pd.read_csv(previousFile, names=col_names, low_memory=False, skiprows=2)
prev_df_size = len(prev_df.index)
print('Done...', prev_df_size)

Reading from previous file...
Done... 536436


In [9]:
# compare the two dataframes
# https://hackersandslackers.com/compare-rows-pandas-dataframes/
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    diff_df.to_csv('expenses/diff.csv')
    return diff_df

In [10]:
df_diff_df = dataframe_difference(df, prev_df)

In [11]:
df_diff_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge
44797,Economic Development and Trade,Board Member,"Danks,Jeannette",Travel,Honorarium,12/08/2017,$319.22,Hotel accommodation for Jeannette Danks attend...,NaN,NaN,NaN,left_only
44798,Economic Development and Trade,NADC Board Member,"Noskey,Ken",Travel,Honorarium,12/08/2017,$319.22,Hotel accommodation for Ken Noskey attending C...,NaN,NaN,NaN,left_only
44804,Economic Development and Trade,Board Member,"Schulz,Jason",Travel,Honorarium,12/08/2017,$319.22,Hotel accommodation for Jason Schulz attending...,NaN,NaN,NaN,left_only
45470,Community and Social Services,Press Secretary,"Toogood,Kathryn",Travel,Ground Transportation,11/22/2017,-$70.48,NaN,http://expenses.alberta.ca/files/2018-04/img_6...,NaN,NaN,left_only
45471,Community and Social Services,Press Secretary,"Toogood,Kathryn",Travel,Ground Transportation,11/30/2017,$140.96,NaN,http://expenses.alberta.ca/files/2018-04/img_6...,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
540270,Infrastructure,Minister,"Panda,Prasad",Travel,Accommodations,10/12/2020,$367.34,Government Government Oct 12th - Coast Edmonto...,NaN,NaN,NaN,right_only
540271,Indigenous Relations,Minister's Chief of Staff,"Bauer,Ted",Travel,Mileage,09/01/2020,$23.10,Backfoot Conf. Protocol sign - Edmonton -Wetas...,NaN,NaN,NaN,right_only
540272,Indigenous Relations,Senior Advisor,"Braun,Riley",Travel,Other Costs,08/16/2020,$15.75,Calgary Stakeholder Meetings - Residence Inn-C...,http://expenses.alberta.ca/files/2020-10/exc_2...,NaN,NaN,right_only
540273,Indigenous Relations,Sr. Executive Advisor CEO AIOC,"Machielse,Matthew",Travel,Accommodations,10/31/2019,$223.42,Calgary Business - Accom. - Delta Hotel,http://expenses.alberta.ca/files/2020-10/exc_6...,NaN,NaN,right_only


In [12]:
#Save diff Df
filename = 'expenses/' + 'diff_' + str( time.strftime('%Y%m%d') ) + ".csv"
df_diff_df.to_csv(filename)

In [13]:
# save file name to filenames table in database
filetosave = Filenames(
        name=filename,
        records = df_size,
        created_at=datetime.now()
    )
try:
    session.add(filetosave)
    session.commit()
except SQLAlchemyError as e:
    session.rollback()
    print('Could not save the new filename %s', filetosave)

Following code cleans the original dataframe df

In [14]:
shift_df = df[df.Date.str.startswith('$')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
58783,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/21/2020,$27.27,Calgary Hearings - Mileage - 54KM,NaN,NaN,NaN,NaN
58784,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/21/2020,$12.75,Calgary Hearings - Parking - RGH,NaN,NaN,NaN,NaN
58785,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,02/28/2020,$30.30,Calgary Hearings - Mileage - 60KM,NaN,NaN,NaN,NaN
58787,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,03/06/2020,$47.47,Calgary Hearings - Mileage - 94KM,NaN,NaN,NaN,NaN
58788,Health,"""Sowa,Bernard, Dr.",Travel,Honorarium,03/06/2020,$12.75,Calgary Hearings - parking - SHC,NaN,NaN,NaN,NaN
66851,Health,"""Juhas,Michal",Travel,Honorarium,02/26/2020,$31.31,Edmonton Hearings - Mileage - 62KM,NaN,NaN,NaN,NaN
66957,Health,"""Juhas,Michal",Travel,Honorarium,03/11/2020,$31.81,Edmonton Hearings - Mileage - 63KM,NaN,NaN,NaN,NaN
77412,Justice and Solicitor General,"""Hak,Elizabeth",Travel,Honorarium,09/05/2019,$318.15,Rauf Hearing in Edmonton - Mileage - Edmonton ...,NaN,NaN,NaN,NaN
77413,Justice and Solicitor General,"""Hak,Elizabeth",Travel,Honorarium,09/05/2019,$20.75,Rauf Hearing in Edmonton - Dinner allowance,NaN,NaN,NaN,NaN
77415,Justice and Solicitor General,"""Hak,Elizabeth",Travel,Honorarium,09/26/2019,$12.75,Parking (no receipt),NaN,NaN,NaN,NaN


In [15]:
len(shift_df)

16

In [16]:
TOTAL_COLS = df.shape[1] -1
for idx, row in shift_df.iterrows():
    new_line = [df.iloc[idx, 0], None]
    for i in df.iloc[idx, 1: TOTAL_COLS]:
        new_line.append(i)
    df.loc[idx] = new_line


In [17]:
# Test
shift_df = df[df.Date.str.startswith('$')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [18]:
shift_df_2 = df[df.Date.str.startswith('H')]
shift_df_2

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
58785,Health,None,NaN,"Sowa,Bernard, Dr.",Travel,Honorarium,02/28/2020,$14.25,Calgary Hearings - Parking - PLC,http://expenses.alberta.ca/files/2020-04/exc_4...,NaN
58788,Health,None,Vice-chair,"Wyrozub,Richard",Travel,Honorarium,02/19/2020,$50.50,Red Deer Hearings - Mileage - 100KM,NaN,NaN
66851,Health,None,Public Member,"Butler,Patricia",Travel,Honorarium,02/19/2020,$85.85,Red Deer Hearings - Mileage - 170KM,NaN,NaN
66957,Health,None,Board Member,"Kreiner,Monica",Travel,Honorarium,02/24/2020,$253.51,Grande Prairie Hearings - Mileage - 502KM,NaN,NaN
77413,Justice and Solicitor General,None,NaN,"Hak,Elizabeth",Travel,Honorarium,09/05/2019,$24.00,Rauf Hearing in Edmonton - Parking,http://expenses.alberta.ca/files/2020-02/exc_1...,NaN
83123,Agriculture and Forestry,None,NaN,"Moore,Norine",Travel,Honorarium,12/13/2019,$114.44,Marketing Council Board Mtg - Accommodation 1 ...,http://expenses.alberta.ca/files/2020-02/exc_B...,NaN
447531,Executive Council,None,NaN,"McLeod,Patricia",Travel,Honorarium,06/17/2016,$117.60,AOE council strategy meeting - Medicine Hat - ...,http://expenses.alberta.ca/files/2016-10/exc_8...,NaN


In [19]:
len(shift_df_2)

7

In [20]:
TOTAL_COLS = df.shape[1] -1
shift_df_2
for idx, row in shift_df_2.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [21]:
# Test
shift_df = df[df.Date.str.startswith('H')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [22]:
shift_df_3 = df[df.Date.str.startswith('O')]
shift_df_3

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
77415,Agriculture and Forestry,None,Exec Director Wildfire Mgt. Br,"Tapp,North Darren James",Travel,Other Costs,11/20/2019,$7.35,Ottawa Trip -,NaN,NaN


In [23]:
len(shift_df_3)

1

In [24]:
TOTAL_COLS = df.shape[1] -1
shift_df_3
for idx, row in shift_df_3.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [25]:
# Test
shift_df = df[df.Date.str.startswith('O')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


In [26]:
shift_df_4 = df[df.Date.str.startswith('M')]
shift_df_4

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
447533,Education,None,Deputy Minister of Education,"Clarke,Curtis",Travel,Meals,10/11/2016,-$9.20,CMEC - Toronto - Breakfast (Adjustment),NaN,NaN


In [27]:
TOTAL_COLS = df.shape[1] -1
shift_df_4
for idx, row in shift_df_4.iterrows():
    new_line = []
    for i in df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df.loc[idx] = new_line

In [28]:
# Test
shift_df = df[df.Date.str.startswith('M')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3


Following code cleans the diff dataframe

In [29]:
shift_df = df_diff_df[df_diff_df.Date.str.startswith('$')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [30]:
len(shift_df)

0

In [31]:
TOTAL_COLS = df_diff_df.shape[1] -1
for idx, row in shift_df.iterrows():
    new_line = [df_diff_df.iloc[idx, 0], None]
    for i in df_diff_df.iloc[idx, 1: TOTAL_COLS]:
        new_line.append(i)
    df_diff_df.loc[idx] = new_line

In [32]:
shift_df_2 = df_diff_df[df_diff_df.Date.str.startswith('H')]
shift_df_2

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [33]:
len(shift_df_2)

0

In [34]:
# Test
shift_df = df_diff_df[df_diff_df.Date.str.startswith('H')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [35]:
TOTAL_COLS =df_diff_df.shape[1] -1
shift_df_2
for idx, row in shift_df_2.iterrows():
    new_line = []
    for i in df_diff_df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df_diff_df.loc[idx] = new_line

In [36]:
# Test
shift_df = df_diff_df[df_diff_df.Date.str.startswith('H')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [37]:
shift_df_3 = df_diff_df[df_diff_df.Date.str.startswith('O')]
shift_df_3

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [38]:
len(shift_df_3)

0

In [39]:
TOTAL_COLS = df_diff_df.shape[1] -1
shift_df_3
for idx, row in shift_df_3.iterrows():
    new_line = []
    for i in df_diff_df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df_diff_df.loc[idx] = new_line

In [40]:
# Test
shift_df = df_diff_df[df_diff_df.Date.str.startswith('O')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [41]:
shift_df_4 = df_diff_df[df_diff_df.Date.str.startswith('M')]
shift_df_4

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


In [42]:
TOTAL_COLS = df_diff_df.shape[1] -1
shift_df_4
for idx, row in shift_df_4.iterrows():
    new_line = []
    for i in df_diff_df.iloc[idx, : TOTAL_COLS+1]:
        new_line.append(i)
    df_diff_df.loc[idx] = new_line

In [43]:
# Test
shift_df = df_diff_df[df_diff_df.Date.str.startswith('M')]
shift_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3,_merge


SAVE DIff dataframe here

The next cells identify around 10000 records that were not saved in the initial run.  Not sure why, probably a DB connection issue

In [ ]:
if run_all_flag:
    df.query('Name == "Filevich,Patricia"')

In [ ]:
if run_all_flag:
    df.loc[509620:509630]

In [ ]:
# new dataframe of missing records
if run_all_flag:
    missing_records_df = df.loc[498839: 509627]

In [ ]:
if run_all_flag:
    missing_records_df.head()

In [ ]:
if run_all_flag:
    missing_records_df.tail()

In [ ]:
if run_all_flag:
    len(missing_records_df)

Duplicates

In [44]:
# identify duplicate values
dups_df= df[df.duplicated(keep=False)] # identifies 850
# dups_df= df[df.duplicated()] # Identifies 431

In [45]:
dups_df

,Ministry,Position,Name,Type,Category,Date,Amount,Description,Receipt 1,Receipt 2,Receipt 3
3045,Justice and Solicitor General,Executive Director,"Merryweather,Lorne",Travel,Ground Transportation,12/13/2013,$3.20,ETS,NaN,NaN,NaN
5094,Culture,Executive Director Arts Branch,"Anderson,Jeffrey",Travel,Mileage,10/10/2012,$1.52,Travel in Edmonton to departmental meeting,NaN,NaN,NaN
5096,Culture,Executive Director Arts Branch,"Anderson,Jeffrey",Travel,Mileage,10/10/2012,$1.52,Travel in Edmonton to departmental meeting,NaN,NaN,NaN
5803,Executive Council,"Exec Dir Comp, Job Eval & Ben","Smith,Debra M",Travel,Air Fare,05/02/2013,$15.00,WestJet Seat Selection,NaN,NaN,NaN
5804,Executive Council,"Exec Dir Comp, Job Eval & Ben","Smith,Debra M",Travel,Air Fare,05/02/2013,$15.00,WestJet Seat Selection,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
537988,Justice and Solicitor General,Assistant Deputy Minister,"Sanderson,Kimberley Anne",Travel,Other Costs,05/11/2020,$12.00,"""",NaN,NaN,NaN
538703,"Culture, Multiculturalism and Status of Women","Executive Director, RAM","Robinson,N Chris",Travel,Other Costs,11/02/2020,$10.00,Parking for meeting,NaN,NaN,NaN
538704,"Culture, Multiculturalism and Status of Women","Executive Director, RAM","Robinson,N Chris",Travel,Other Costs,11/02/2020,$10.00,Parking for meeting,NaN,NaN,NaN
538868,Agriculture and Forestry,Chief of Staff,"Schultz,Tim",Travel,Mileage Allowance,11/12/2020,$151.50,Meetings & Tour in Calgary,NaN,NaN,NaN


In [ ]:
# FUTURE only process delta dataframe!!!


# Process the dataframe
# Note that after the very first attempt at this we need to test for whether the record already exist and then update the
# date last found

start_time = time.time()
length = len(df)
ctr = 0

for index, row in df.iterrows():
    
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
    newAmount = df._get_value(index, 'Amount').replace('$', '').replace(',','')
    newAmount = float(newAmount)

    if len(str(df._get_value(index, 'Receipt 1'))) > 5:
        newReceipt1 = df._get_value(index, 'Receipt 1')
    else:
        newReceipt1 = ''
    if len(str(df._get_value(index, 'Receipt 2'))) > 5:
        newReceipt2 = df._get_value(index, 'Receipt 2')
    else:
        newReceipt2 = ''
    if len(str(df._get_value(index, 'Receipt 3'))) > 5:
        newReceipt3 = df._get_value(index, 'Receipt 3')
    else:
        newReceipt3 = ''  
    
    expense = ExpensesRaw(
        ministry=df._get_value(index, 'Ministry'),
        position=df._get_value(index, 'Position'),
        name=df._get_value(index, 'Name'),
        type=df._get_value(index, 'Type'),
        category=df._get_value(index, 'Category'),
        expense_date=newdate,
        amount=newAmount,
        description=df._get_value(index, 'Description'),
        receipt1=newReceipt1,
        receipt2=newReceipt2,
        receipt3=newReceipt3,
        date_last_found=datetime.now(),
        created_at=datetime.now(),
        updated_at=datetime.now(),
    )
    
#     qry = session.query(ExpensesRaw)
#     qry = qry.filter(
#                      ExpensesRaw.ministry==df._get_value(index, 'Ministry'),
#                      ExpensesRaw.position==df._get_value(index, 'Position'),
#                      ExpensesRaw.name==df._get_value(index, 'Name'),
#                      ExpensesRaw.type==df._get_value(index, 'Type'),
#                      ExpensesRaw.category==df._get_value(index, 'Category'),
#                      ExpensesRaw.expense_date==newdate,
#                      ExpensesRaw.amount==newAmount,
#                      ExpensesRaw.description==df._get_value(index, 'Description'),
#                      ExpensesRaw.receipt1==df._get_value(index, 'Receipt 1'),
#                      ExpensesRaw.receipt2==df._get_value(index, 'Receipt 2'),
#                      ExpensesRaw.receipt3==df._get_value(index, 'Receipt 3')
#                     ).first()
#     print('Query', qry)
    
    if ctr % 100 == 0:
        perc = "{:.3f}".format(ctr/length)
        elapsed = "{:.2f}".format(time.time() - start_time)
        print('Count: %s of %s percentage %s elapsed %s ' %(ctr, length, perc, elapsed))
    
    ctr = ctr + 1
    
    try:
        session.add(expense)
        session.commit()
    except SQLAlchemyError as e:
#         error = str(e.__dict__['orig'])
        session.rollback()
        print('Could not save the new expense %s',
              expense)
#         print(error)
   

In [ ]:
# for each row in dups_df find it in the expensesraw table of the db and update the isduplicated flag to true
ctr = 1
for index, row in dups_df.iterrows():
    #find the row in the database
    newdate = df._get_value(index, 'Date')
    newdate = datetime.strptime(newdate, '%m/%d/%Y')
    newAmount = df._get_value(index, 'Amount').replace('$', '').replace(',','')
    newAmount = float(newAmount)

    if len(str(df._get_value(index, 'Receipt 1'))) > 5:
        newReceipt1 = df._get_value(index, 'Receipt 1')
    else:
        newReceipt1 = ''
    if len(str(df._get_value(index, 'Receipt 2'))) > 5:
        newReceipt2 = df._get_value(index, 'Receipt 2')
    else:
        newReceipt2 = ''
    if len(str(df._get_value(index, 'Receipt 3'))) > 5:
        newReceipt3 = df._get_value(index, 'Receipt 3')
    else:
        newReceipt3 = ''
        
    print('Position : ',  df._get_value(index, 'Position'))
    newPosition = df._get_value(index, 'Position')
    
    if df.isnull()._get_value(index, 'Position'):
        print('Position is None')
        newPosition = ''
        
    print('Description : ',  df._get_value(index, 'Description'))
    newDescription = df._get_value(index, 'Description')
    
    if df.isnull()._get_value(index, 'Description'):
        print('Description is None')
        newDescription = ''    
    
    qry = session.query(ExpensesRaw)
    qry = qry.filter(
                     ExpensesRaw.ministry==df._get_value(index, 'Ministry'),
                     ExpensesRaw.position==newPosition,
                     ExpensesRaw.name==df._get_value(index, 'Name'),
                     ExpensesRaw.type==df._get_value(index, 'Type'),
                     ExpensesRaw.category==df._get_value(index, 'Category'),
                     ExpensesRaw.expense_date==newdate,
                     ExpensesRaw.amount==newAmount,
                     ExpensesRaw.description==newDescription,
                     ExpensesRaw.receipt1==newReceipt1,
                     ExpensesRaw.receipt2==newReceipt2,
                     ExpensesRaw.receipt3==newReceipt3,
                    ).all()
    for result in qry:
        result.is_duplicated = True
#         print(result)
    try:
        session.commit()
    except:
        print('Could not update')
    print('Count', ctr)
    ctr = ctr + 1    
    

In [ ]:
print(df.head())